In [1]:
def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 100.0 * currentCount / totalCount
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\r[{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print('\n')


def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = len(lst_long) / n
    else:
        totalBatches = len(lst_long) / n + 1
    for i in range(int(totalBatches)):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted

In [2]:
import sys,random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

In [3]:
from sqlalchemy import create_engine
class SteamObject(object):
    
    def config_database_MySQLConnect(self, username,password,host,database):
        engine = create_engine('mysql+pymysql://{}:{}@{}/{}?charset=utf8mb4'.format(username, password, 
                                                                                    host, database))
        return engine
        
    def MySQL_query(self, query):
        df = pd.read_sql_query(query,engine)
        return df
    

In [8]:
# enter your username,password, host and database in mySQL
Steam = SteamObject()
# config database connection
username = 'root'
password = 'Brady811215'
host = '127.0.0.1'
database = 'game_recommendation'
engine = Steam.config_database_MySQLConnect(username,password,host,database)

In [9]:
query2 = '''SELECT user_id,app_id, playtime_forever FROM game_steam_user_inventory where playtime_forever <> 0'''
df_game_time = Steam.MySQL_query(query2)
df_game_time.head()

,user_id,app_id,playtime_forever
0,76561198158086086,4000,370
1,76561198158086086,110800,9
2,76561198158086086,250320,507
3,76561198158086086,296470,78
4,76561198158086086,301520,20


In [11]:
dic_purchase = {}
set_valid_app_id = set([i[0] for i in engine.execute('SELECT app_id FROM game_steam_app WHERE short_description IS NOT NULL AND type = "game"AND release_date <= CURDATE() AND initial_price IS NOT NULL').fetchall()])
for app_id, user_id in engine.execute('SELECT app_id, user_id FROM game_steam_user_inventory').fetchall():
    if app_id in set_valid_app_id:
        if user_id in dic_purchase:
            dic_purchase[user_id].update({app_id : 1})
        else:
            dic_purchase[user_id] = {app_id : 1}

In [13]:
df_purchase = pd.DataFrame(dic_purchase).fillna(0)
purchase_matrix = df_purchase.values
lst_app_id = df_purchase.index

In [17]:
purchase_matrix[:1]

array([[0., 0., 0., ..., 0., 0., 0.]])

In [23]:
def recommendation_item_based():
    print('Item Based Model')
    dic_purchase = {}
    set_valid_app_id = set([i[0] for i in engine.execute('SELECT app_id FROM game_steam_app WHERE short_description IS NOT NULL AND type = "game"AND release_date <= CURDATE() AND initial_price IS NOT NULL').fetchall()])
    for app_id, user_id in engine.execute('SELECT app_id, user_id FROM game_steam_user_inventory').fetchall():
        if app_id in set_valid_app_id:
            if user_id in dic_purchase:
                dic_purchase[user_id].update({app_id : 1})
            else:
                dic_purchase[user_id] = {app_id : 1}

    df_purchase = pd.DataFrame(dic_purchase).fillna(0)
    purchase_matrix = df_purchase.values
    lst_app_id = df_purchase.index

    total_count = purchase_matrix.shape[0]
    current_count = 0

    dic_recomended_item_based = {}
    for index in range(total_count):
        cosine_similarities = linear_kernel(purchase_matrix[index:index+1], purchase_matrix).flatten()
        lst_related_app = np.argsort(-cosine_similarities)[1:21]
        dic_recomended_item_based.update({lst_app_id[index]:[lst_app_id[i] for i in lst_related_app]})
        show_work_status(1,total_count,current_count)
        current_count+=1


    df_item_based_result = pd.DataFrame.from_dict(dic_recomended_item_based, 'index')
    df_item_based_result.index.name = 'app_id'
    df_item_based_result.reset_index(inplace=True)
    df_item_based_result.to_sql('recommended_games_item_based',engine,if_exists='replace', index = False)
    return df_item_based_result
df_item_based_result = recommendation_item_based()
    

Item Based Model
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [24]:
df_item_based_result

,app_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,4000,730,352460,304930,550,218620,105600,230410,240,301520,...,252950,620,72850,252490,49520,224260,236390,333930,304050,271590
1,110800,4000,730,550,352460,218620,72850,105600,620,240,...,304930,340,8190,219640,33930,220,271590,12210,252950,8870
2,250320,4000,550,352460,730,218620,72850,49520,620,105600,...,322330,8870,340,1250,219640,304930,55230,252950,8930,203160
3,296470,4000,730,352460,550,304930,218620,105600,240,620,...,322330,49520,72850,219640,252490,230410,271590,238460,207140,1250
4,301520,304930,730,4000,352460,550,230410,218620,227940,236390,...,105600,224260,333930,218230,240,252490,252950,72850,620,49520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6554,394220,240,393420,220240,72850,244850,386360,49520,204360,105600,...,304930,252490,339600,427520,252870,273110,550,298240,355180,404410
6555,313330,100,80,10,253900,72850,299360,320,49520,270550,...,265930,96100,226120,333930,254060,291480,230410,8190,48700,292390
6556,371550,330070,252170,241930,33520,33530,255280,332800,299460,105000,...,97000,313690,233130,6830,355840,234670,295690,105600,262240,204360
6557,357690,399230,6900,6860,6850,239430,428690,11370,433850,411560,...,274920,359960,264710,402330,397460,206420,265210,218640,246900,241930


In [46]:
def hit_rate_metric():
    user_id = df_game_time.groupby('user_id').max().reset_index().user_id.tolist()
    app_id = df_game_time.groupby('user_id').max().reset_index().app_id.tolist()
    df = df_ratings.loc[df_ratings['rating']>0]
    hit = 0
    for app,user in zip(app_id,user_id):
        recommended_lst = df_item_based_result.loc[df_item_based_result['app_id']==app].values.flatten().tolist()[:10]
        for rec in recommended_lst:
            if rec in df.loc[df['user_id']==user].app_id.unique().tolist():
                hit +=1
    hit_rate = hit/len(user_id)
    return hit_rate
                
        

In [47]:
hit_rate_metric()

5.53427144802542